In [59]:
import pandas as pd
import numpy as np

import yfinance as yf

In [60]:
TICKS = [
    'SPY',
    'VTV',
    'AGG',
    'HYG',
    'IEF',
    'PSP',
    'APO',
    'BX',
    'SHV'
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [61]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [62]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,27128200,623795240960,The Trust seeks to achieve its investment obje...
VTV,Vanguard Value ETF,ETF,USD,1419036,187688484864,The fund employs an indexing investment approa...
AGG,iShares Core U.S. Aggregate Bon,ETF,USD,5223168,119716577280,The index measures the performance of the tota...
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,30111422,14676305920,The underlying index is a rules-based index co...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,4221735,31385524224,The underlying index measures the performance ...
PSP,Invesco Global Listed Private E,ETF,USD,7241,276040096,The fund generally will invest at least 90% of...
APO,"Apollo Global Management, Inc.",EQUITY,USD,2035262,NaN,"Apollo Global Management, Inc. is a private eq..."
BX,Blackstone Inc.,EQUITY,USD,1540734,NaN,Blackstone Inc. is an alternative asset manage...
SHV,iShares Short Treasury Bond ETF,ETF,USD,1430549,19008184320,The fund will invest at least 80% of its asset...


In [63]:
STARTDATE = '2015-01-01'
ENDDATE = '2024-12-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE, auto_adjust=False)['Adj Close']

[*********************100%***********************]  9 of 9 completed


In [64]:
#prices = df.resample('M').last()
AGG = 'W-FRI'
prices = df.resample(AGG).last().iloc[:-1,:]
#prices = df

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

In [65]:
rets

Ticker,AGG,APO,BX,HYG,IEF,PSP,SHV,SPY,VTV
Date,,,,,,,,,
2015-01-09,0.005433,0.012728,0.002344,0.005579,0.013517,-0.007306,0.000182,-0.005744,-0.007218
2015-01-16,0.004233,0.003351,-0.015785,-0.005882,0.010188,-0.013799,0.000181,-0.012828,-0.011681
2015-01-23,0.001883,0.027975,0.070983,0.004913,0.001559,0.019589,0.000000,0.016565,0.011940
2015-01-30,0.005998,0.010155,0.035496,0.002444,0.011992,-0.000915,-0.000091,-0.026931,-0.034323
2015-02-06,-0.010195,-0.022920,-0.013050,0.011962,-0.022723,0.027473,0.000091,0.030584,0.037887
...,...,...,...,...,...,...,...,...,...
2024-11-29,0.014211,0.044581,-0.039990,0.006919,0.016987,0.010151,0.000906,0.011822,0.009940
2024-12-06,0.001311,0.016112,-0.021404,-0.001749,0.000840,-0.001814,-0.002624,0.008730,-0.019629
2024-12-13,-0.014195,-0.007984,0.007594,-0.006133,-0.016584,-0.016359,0.000726,-0.005923,-0.018172


In [66]:
retsx

Ticker,AGG,APO,BX,HYG,IEF,PSP,SPY,VTV
Date,,,,,,,,
2015-01-09,0.005252,0.012546,0.002162,0.005398,0.013335,-0.007488,-0.005925,-0.007400
2015-01-16,0.004052,0.003170,-0.015966,-0.006063,0.010007,-0.013980,-0.013009,-0.011862
2015-01-23,0.001883,0.027975,0.070983,0.004913,0.001559,0.019589,0.016565,0.011940
2015-01-30,0.006088,0.010245,0.035587,0.002534,0.012083,-0.000824,-0.026840,-0.034232
2015-02-06,-0.010286,-0.023011,-0.013141,0.011872,-0.022814,0.027382,0.030493,0.037797
...,...,...,...,...,...,...,...,...
2024-11-29,0.013306,0.043675,-0.040896,0.006013,0.016082,0.009245,0.010916,0.009034
2024-12-06,0.003935,0.018736,-0.018779,0.000875,0.003465,0.000810,0.011354,-0.017005
2024-12-13,-0.014921,-0.008710,0.006868,-0.006859,-0.017310,-0.017085,-0.006649,-0.018898


In [67]:
with pd.ExcelWriter('../data/pe_rep_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')